# Simulation of Moon TOD using real scan data

Not possible until the file "CalQubic_Synthbeam_Analytical_150_TD.fits" exists!

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib ipympl


from datetime import datetime, timezone
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import astropy_mpl_style, quantity_support
# plt.style.use(astropy_mpl_style)
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_body

# plt.rc('figure',figsize=(20,12))
plt.rc('figure',figsize=(10,7))
plt.rc('font',size=16)


Salta = EarthLocation(lat=-24.731375*u.deg, lon=-65.409551*u.deg, height=1152*u.m)
####utcoffset = -3*u.hour  # Eastern Daylight Time

from scipy.signal import medfilt
from scipy.interpolate import interp1d
import glob
import gc
import pickle

from qubicpack.qubicfp import qubicfp
from qubicpack.iv import ADU2I
import qubic.lib.Calibration.Qfiber as ft
import qubic
import healpy as hp
from qubicpack.utilities import Qubic_DataDir
from qubic.lib import Qdictionary, Qscene, Qinstrument, Qacquisition, Qsamplings
from qubic.lib.MapMaking.Qcg import pcg
from pyoperators import DiagonalOperator
from qubic.lib.Qfoldertools import do_gif
from pyoperators import MPI

from importlib import reload
import healpy as hp

import time_domain_tools as tdt

import pipeline_moon_functions as pmf
import pipeline_moon_plotting as pmp


Let's first read some QUBIC TOD in order to get the azimuth/elevation. We take the data for the Moon scan taken on 14/07/2022.
The zero azimuth for QUBIC that day was: azqubic = 116.4

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'
thedate = '2022-07-14'
thedirs = glob.glob(mydatadir + '/' + thedate + '/*')
thedata = thedirs[0]
azqubic = 116.4


### Read data
a = qubicfp()
a.read_qubicstudio_dataset(thedata)

### We don;t need to fill the memory with the TODs
#tt, alltod = a.tod()

az = a.azimuth() + azqubic
# az = -a.azimuth() + azqubic # testing a reverted azimuth --> not working, if the starting point's azimuth is well-known, then if we were going the wrong direction we wouldn't see the Moon!
el = a.elevation()
thk = a.timeaxis(datatype='hk')

#del(a)

### We remove tt[0]
tinit = thk[0]
thk -= tinit

# plt.figure()
# plt.subplot(1,3,1)
# plt.plot(az, el)
# plt.xlabel('az')
# plt.ylabel('el')

# plt.subplot(1,3,2)
# plt.plot(thk, az)
# plt.xlabel('t')
# plt.ylabel('az')

# plt.subplot(1,3,3)
# plt.plot(thk, el)
# plt.xlabel('t')
# plt.ylabel('el')
# plt.show()

In [ ]:
del(a)
gc.collect() 

In [ ]:
### Create a QubicSampling object from these

### Don't forget the UTC offset
####utcoffset = 3.

date_obs = str(datetime.utcfromtimestamp(tinit))
print('Observations started at: {} UTC (old)'.format(date_obs))

# better way but the output format needs to be well understood in the following functions such as QubicSampling
# date_obs = str(datetime.fromtimestamp(tinit, timezone.utc))
# print('Observations started at: {} UTC'.format(date_obs))

qs = Qsamplings.QubicSampling(azimuth=az, elevation=el, time=thk,
# qs = Qsamplings.QubicSampling(azimuth=el, elevation=az, time=thk, # testing swapping azimuth and elevation to see how it affects the maps and the orientation of the synthetic beam --> not working if the Moon coord not changed?
                   #period=np.median(thk[1:]-thk[:-1]), 
                   date_obs=date_obs, longitude=float(Salta.lon/u.deg), latitude = float(Salta.lat/u.deg))

In [ ]:
%%script echo skipping
# %matplotlib inline
plt.figure()
plt.subplot(2,3,1)
plt.plot(qs.time, qs.equatorial[:,0])
plt.xlabel('UTC Time from QS (sec)')
plt.ylabel('RA from QS (deg)')
plt.subplot(2,3,2)
plt.plot(qs.time , qs.equatorial[:,1])
plt.xlabel('UTC Time from QS (sec)')
plt.ylabel('DEC from QS (deg)')
plt.subplot(2,3,3)
plt.plot(qs.equatorial[:,0], qs.equatorial[:,1])
plt.xlabel('RA from QS (deg)')
plt.ylabel('DEC from QS (deg)')
plt.tight_layout()

plt.subplot(2,3,4)
plt.plot(qs.azimuth, qs.elevation, label='From QS')
plt.plot(az, el, '--', label='From HK')
plt.xlabel('az (deg)')
plt.ylabel('el (deg)')

plt.subplot(2,3,5)
plt.plot(qs.time, qs.azimuth, label='From QS')
plt.plot(thk, az, '--', label='From HK')
plt.xlabel('time (sec)')
plt.ylabel('az (deg)')

plt.subplot(2,3,6)
plt.plot(qs.time, qs.elevation, label='From QS')
plt.plot(thk, el, '--', label='From HK')
plt.xlabel('time (sec)')
plt.ylabel('el (deg)')
plt.show()


We now need the coordinates of the Moon at these very same time samples. We use astropy for that purpose.

In [ ]:

alltimes = Time(date_obs) + thk*u.second #+ 3*u.minute

############################### Using Astropy #########################
### Local coordinates
frame_Salta = AltAz(obstime=alltimes , location=Salta)

### Moon
moon_Salta = get_body('Moon', alltimes, Salta)

### RA/DEC
moonra = moon_Salta.ra
moondec = moon_Salta.dec

# Computing the mean angular size of the Moon
moon_dist = moon_Salta.distance.value * pmf.AU_meters #m
moon_size = 2 * 1737.4 * 1e3 #m
moon_angsize = np.mean(np.arctan(moon_size/moon_dist)) # radians
print("Moon angular size = {} degree".format(moon_angsize * 180/np.pi))

### l,b
g = SkyCoord(moon_Salta.ra, moon_Salta.dec, frame='icrs').galactic
moongal_l, moongal_b = g.l.degree, g.b.degree
# moongal_l, moongal_b = qubic.equ2gal(moonra, moondec)

moonaltazs_Salta = moon_Salta.transform_to(frame_Salta)  
myazmoon = moonaltazs_Salta.az.value
myelmoon = moonaltazs_Salta.alt.value

### Now we know the equatorial/Galactic location of the moon 
lmoon_av = np.mean(np.array(moongal_l))
bmoon_av = np.mean(np.array(moongal_b))

ramoon_av = np.mean(moonra)
decmoon_av = np.mean(moondec)
########################################################################

In [ ]:
%%script echo skipping
plt.figure()
plt.subplot(2,2,1)
plt.plot(qs.azimuth, qs.elevation, label='QS')
plt.plot(myazmoon, myelmoon, 'ro', label='Astropy')
plt.legend()
plt.xlabel('az')
plt.ylabel('el')

plt.subplot(2,2,2)
plt.plot(qs.azimuth-myazmoon, qs.elevation-myelmoon, label='QS')
plt.plot(myazmoon - myazmoon, myelmoon-myelmoon, 'ro', label='Astropy')
plt.legend()
plt.xlabel('az - azmoon')
plt.ylabel('el-elmoon')


plt.subplot(2,2,3)
plt.plot(qs.equatorial[:,0], qs.equatorial[:,1], label='QS')
plt.plot(moonra, moondec, 'ro', label='Astropy')
plt.plot(ramoon_av, decmoon_av, 'y+', ms=20, mew=3, label='Mean Astropy')
plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('DEC (deg)')
plt.tight_layout()

plt.subplot(2,2,4)
plt.plot(qs.galactic[:,0], qs.galactic[:,1], label='QS')
plt.plot(moongal_l, moongal_b, 'ro', label='Astropy')
plt.plot(lmoon_av, bmoon_av, 'y+', ms=20, mew=3, label='Mean Astropy')
plt.legend()
plt.xlabel('l (deg)')
plt.ylabel('b (deg)')
plt.tight_layout()


# Now trying to get the Moon moving
For this we need to be in a coordinate system where the Moon does not move, then perform acquisition in this system and use this TOD in the former system (that of the real data).


## method:
Calculate Moon coordinates in (RA,DEC) or (l,b) using astropy, subtract this to pointing's sky coordinates and come back to Az,el...

In [ ]:
### Shift in RA,DEC
shiftra = (moonra - np.mean(moonra))/u.deg
shiftdec = (moondec - np.mean(moondec))/u.deg
newra = (qs.equatorial[:,0] - shiftra) 
newdec = qs.equatorial[:,1] - shiftdec

newmoonra = moonra/u.deg - shiftra
newmoondec = moondec/u.deg - shiftdec

### COnvert to Gal coordinates
newl, newb = Qsamplings.equ2gal(newra, newdec)
newmoonl, newmoonb = Qsamplings.equ2gal(newmoonra, newmoondec)

# deltazt, deltelt = 4.5658359258853585, 1.4750926931640054
deltazt, deltelt = 0, 0

### Now we need to got to local coordinates az,el...
altaz = SkyCoord(newra*u.deg, newdec*u.deg, frame='icrs').transform_to(frame_Salta) 
newaz = altaz.az.value + deltazt # trying to shift by hand the pointings
newel = altaz.alt.value + deltelt
altazmoon = SkyCoord(newmoonra*u.deg, newmoondec*u.deg, frame='icrs').transform_to(frame_Salta) 
newmoonaz = altazmoon.az.value
newmoonel = altazmoon.alt.value

### New sampling
qsmoon = Qsamplings.QubicSampling(azimuth=newaz, elevation=newel, time=qs.time,
                   date_obs=date_obs, longitude=float(Salta.lon/u.deg), latitude = float(Salta.lat/u.deg),
                   pitch=5) # 5 is close to fit?
                       
# plt.figure()
# plt.subplot(2,3,2)
# plt.plot(qsmoon.galactic[:,0], qsmoon.galactic[:,1], label='New QSmoon')
# plt.plot(qs.galactic[:,0], qs.galactic[:,1], label='Initital QS', alpha=0.5)
# plt.plot(newmoonl, newmoonb, 'ro', label='Moon Motion')
# plt.xlabel('New l')
# plt.ylabel('New b')
# plt.legend()

# plt.subplot(2,3,1)
# plt.plot(qsmoon.equatorial[:,0], qsmoon.equatorial[:,1], label='New QSmoon')
# plt.plot(qs.equatorial[:,0], qs.equatorial[:,1], label='Initital QS', alpha=0.5)
# plt.plot(newmoonra, newmoondec, 'ro', label='Moon Motion')
# plt.xlabel('New RA')
# plt.ylabel('New DEC')
# plt.legend()


# plt.subplot(2,3,3)
# plt.plot(qsmoon.azimuth, qsmoon.elevation, label='New QSmoon')
# plt.plot(qs.azimuth, qs.elevation, label='Initital QS', alpha=0.5)
# plt.plot(newmoonaz, newmoonel, 'ro', label='Moon Motion')
# plt.xlabel('New az')
# plt.ylabel('New el')
# plt.legend()
# plt.show()


In [ ]:
%%script echo skipping

plt.figure()
plt.subplot(1,3,1)
plt.plot(qsmoon.azimuth, qsmoon.elevation, label='QSmoon')
plt.plot(newmoonaz, newmoonel, 'ro', label='Moon Motion')
plt.xlabel('(az - az_moon)')
plt.ylabel('el - el_moon')
plt.legend()

plt.subplot(1,3,2)
plt.plot(qsmoon.equatorial[:,0], qsmoon.equatorial[:,1], label='QSmoon')
plt.plot(newmoonra, newmoondec, 'ro', label='Moon Motion')
plt.xlabel('RA From Moon')
plt.ylabel('DEC From Moon')
plt.legend()

plt.subplot(1,3,3)
plt.plot(qsmoon.galactic[:,0], qsmoon.galactic[:,1], label='QSmoon')
plt.plot(newmoonl, newmoonb, 'ro', label='Moon Motion')
plt.xlabel('Gal l From Moon')
plt.ylabel('Gal b From Moon')
plt.legend()
plt.show()


In [ ]:
### Now we know the equatorial/Galactic location of the moon in this referential attached to the Moon. We can create an image of the moon there
nside = 256

lmoon_av = np.mean(newmoonl)  ### Mean but in fact it is constant
bmoon_av = np.mean(newmoonb)  ### Mean but in fact it is constant


print(lmoon_av, bmoon_av)
uvmoon = hp.ang2vec(np.radians(90.-bmoon_av), np.radians(lmoon_av))

allip = np.arange(12*nside**2)
pixmoon = hp.query_disc(nside, uvmoon, moon_angsize, inclusive=True)
map_in = np.zeros(12*nside**2)
map_in[pixmoon] = 1

plt.figure()
hp.mollview(map_in, sub=(2,2,1))
hp.gnomview(map_in, rot=[lmoon_av, bmoon_av], reso=10, sub=(2,2,2))

### We can calculate the coverage from the l,b of the pointing and display it
ipcov = hp.ang2pix(nside, np.radians(90.-qsmoon.galactic[:,1]), np.radians(qsmoon.galactic[:,0]))
mapcov = map_in.copy()
mapcov[ipcov] += 1
hp.mollview(mapcov, sub=(2,2,3))
hp.gnomview(mapcov, rot=[lmoon_av, bmoon_av], reso=10, sub=(2,2,4))
plt.show()


In [ ]:
def get_TOD(d, qsmoon, y0):
    ### We use our fake moon-frame pointing qsmoon
    qsmoon.fix_az = False
    # a = Qacquisition.QubicMultiAcquisitions(q, qsmoon, s, d, nus_edge_in)
    a = Qacquisition.QubicMultiAcquisitions(d, nsub=d['nf_sub'], nrec=2, comps=[], H=None, nu_co=None, sampling=qsmoon) # because QAcquisition not fixed on this branch
    TOD_nsub = []
    for ind_a, a_i in enumerate(a.subacqs):
        TOD_i, mapconv_i = a_i.get_observation(y0[ind_a, :], noiseless=d['noiseless'], convolution = True)
        if ind_a == 0:
            TOD = TOD_i.copy()
        else:
            TOD += TOD_i
        TOD_nsub.append(TOD_i)
    H = a.H.copy()
    del a
    gc.collect()   ### Important ! Frees the memory
    return TOD, H, TOD_nsub

In [ ]:
### Let's simulate TOD from this
# Repository for dictionary and input maps
global_dir = Qubic_DataDir(datafile='instrument.py', datadir='../')
dictfilename = global_dir + '/dicts/pipeline_MoonSalta.dict'
d = Qdictionary.qubicDict()
d.read_from_file(dictfilename)
d['nside'] = 256
d['config'] = 'TD' # no calfile for TD, or even for FI at 150 GHz
d['kind'] = 'I'
d['nf_sub'] = 12 # monoband not implemented on this branch
d['nf_recon'] = 2
d['MultiBand'] = True
d['sweeping_pointing_deadtime'] = False
d['noiseless'] = True
d["use_synthbeam_fits_file"] = False
d["n_nu_fit"] = d['nf_sub']//2 # monoband not implemented on this branch

d['detector_tau'] = 0.01

s = Qscene.QubicScene(d)
q = Qinstrument.QubicMultibandInstrument(d)
# positions = q[0].detector.center
# plt.figure()
# plt.plot(positions[:,0], positions[:,1], 'ro')
# q[0].detector.plot()
# plt.show()

_, nus_edge_in, nus_in, _, _, _ = Qinstrument.compute_freq(d['filter_nu'] / 1e9, d['n_nu_fit'],
                                                d['filter_relative_bandwidth'])
# Create an input map with the moon in each sub-frequency
y0 = np.empty((d['n_nu_fit'], 12 * d['nside'] ** 2))
for i in range(d['n_nu_fit']):
    y0[i,:] = map_in# * (150/nus_in[i])**4

TOD, H, TOD_nsub = get_TOD(d, qsmoon, y0)

In [ ]:
print(d)

In [ ]:
print(np.shape(thk))
print(np.shape(TOD))
print(np.shape(TOD_nsub))

In [ ]:
plt.figure()
plt.plot(thk, TOD[0,:])
plt.show()

In [ ]:
from pyoperators import Cartesian2SphericalOperator
positions = q[0].detector.center
plt.figure()
plt.plot(positions[:,0], positions[:,1], 'ro')
q[0].detector.plot()

uvecs = positions / np.sqrt(np.sum(positions ** 2, axis=-1))[..., None]
sh = np.shape(uvecs)
ths = np.zeros(sh[0])
phs = np.zeros(sh[0])
for i in range(sh[0]):
    ths[i], phs[i] = Cartesian2SphericalOperator('zenith,azimuth')(uvecs[i,:])

plt.figure()
plt.subplot(1,3,1, projection="hammer")
plt.grid()
plt.plot(phs, np.pi/2-ths, 'k.')

rotmat = hp.Rotator(rot=[0, 90, 0], deg=True)
newths, newphs = rotmat(ths, phs)

plt.subplot(1,3,2, projection="hammer")
plt.grid()
plt.plot(newphs-np.pi, np.pi/2-newths, 'k.')

plt.subplot(1,3,3)
plt.plot(np.degrees(newphs), np.degrees(newths), 'ro')

In [ ]:
TESNum = 163 #126 #19 #179 #67 #163 #67 #96, 209, 67, 152, 209     #33->154, 96->231
iTES = TESNum - 1

# TESNum_flip = 163
# iTES_flip = TESNum_flip - 1

indexQS = pmf.iQP2iQS(TESNum-1)
print(indexQS)

nside = 256

### Define the detector positions on the map
pos_det_thetas = newths #np.pi-newths #positions[:,0]
pos_det_phis = np.pi - newphs #-(newphs-np.pi) #positions[:,1]

dth_tes = np.degrees(np.pi/2 - pos_det_thetas[indexQS])
dph_tes = np.degrees(pos_det_phis[indexQS])

In [ ]:
tod = TOD[indexQS,:]

speedmin = 0.05


###### Pipeline:
# 0. Identify scan types and numbers
scantype_hk, azt, elt, scantype, vmean = tdt.identify_scans(thk, qs.azimuth, qs.elevation, tt=qs.time, doplot=False, thr_speedmin=speedmin)
nscans = np.max(np.abs(scantype))

# # Map-making
# mapsb, mapcount = healpix_map(qs.azimuth[scantype != 0], qs.elevation[scantype != 0], tod[scantype != 0], nside=nside)

# # Display Results
# hp.gnomview(mapsb, rot=[np.mean(qs.azimuth), np.mean(qs.elevation)], reso=8, sub=(1,3,1), title='Az/El Co-added Map [QP #{} ; QS #{}]'.format(TESNum, indexQS+1))


# Map-making
newazt = (qs.azimuth - myazmoon) * np.cos(np.radians(qs.elevation))
newelt = (qs.elevation - myelmoon)
# newazt, newelt = pmf.get_new_azel_v2(qs.azimuth, qs.elevation, myazmoon, myelmoon)
# newazt, newelt = pmf.get_new_azel_v4(qs.azimuth, qs.elevation, myazmoon, myelmoon)
mapsb, mapcount = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], tod[scantype != 0], nside=nside)



# Display Results
rot = [0, 0]
# rot = [-0.701, 7.459]
# rot = moon_fit_sim[iTES][0]
plt.figure()
hp.gnomview(mapsb, reso=10, rot=rot, title='Az/El wrt Moon FP centered [QP #{} ; QS #{}]'.format(TESNum, indexQS+1))
# hp.gnomview(mapsb, reso=10, rot=[0, 0], sub=(1,2,1), title='Az/El wrt Moon FP centered [QP #{} ; QS #{}]'.format(TESNum, indexQS+1))


# ### Centered on L.O.S
# hp.gnomview(mapsb, reso=10, rot=[dph_tes, dth_tes], sub=(1,2,2), title='L.O.S. centered [QP #{} ; QS #{}]'.format(TESNum, indexQS+1))
# hp.graticule()
# hp.projplot(np.radians(90),np.radians(0),'r+', mew=3, ms=10, label='FP center')
# hp.projplot(pos_det_thetas, pos_det_phis,'r.', label='detectors')
# hp.projplot(pos_det_thetas[indexQS], pos_det_phis[indexQS],'go', label='QP#{}'.format(TESNum))
# plt.legend()
plt.show()

# The second brightest closer peak seems to not be in the right direction with respect to data, is it possible that the beam is flipped? see TES 67 for very clear example

In [ ]:
# Here I will superimpose the theoretical synthbeam with the simulated to data to see if I obtain the same shift as in the real data (order 1 is not as far from order 0 as in theory)
# The necessary functions are in popeline_moon_functions
# Do the same method as in pipeline_moon_fit_order1_peak.ipynb

In [ ]:
mapsb, mapcount = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], tod[scantype != 0], nside=nside)
mapsb_pos, _ = pmf.healpix_map(newazt[scantype > 0], newelt[scantype > 0], tod[scantype > 0], nside=nside)
mapsb_neg, _ = pmf.healpix_map(newazt[scantype < 0], newelt[scantype < 0], tod[scantype < 0], nside=nside)

# Display Results
anatype = 'Moon Sim'
# display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype, rot=[np.mean(newazt), np.mean(newelt)], reso=10, 
#             myrange=[-6e-25, 3e-22], TESNum=TESNum)

plt.figure()
unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)
mapdiff = mapsb_pos-mapsb_neg
mapdiff[unseen] = hp.UNSEEN
hp.gnomview(mapsb, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,1), title='Both scans QP#{}'.format(TESNum), min=0, max=np.max(mapsb[~unseen]))
hp.gnomview(mapsb_pos, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,2), title='+ scans QP#{}'.format(TESNum), min=0, max=np.max(mapsb[~unseen]))
hp.gnomview(mapsb_neg, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,3), title='- scans QP#{}'.format(TESNum), min=0, max=np.max(mapsb[~unseen]))
hp.gnomview(mapdiff, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,4), title='Diff scans QP#{}'.format(TESNum))

plt.tight_layout()
plt.show()

In [ ]:
maxsb = np.max(mapsb[~unseen])

newmapsb = np.zeros(12*nside**2)+hp.UNSEEN
newmapsb_pos = np.zeros(12*nside**2)+hp.UNSEEN
newmapsb_neg = np.zeros(12*nside**2)+hp.UNSEEN

unseen_sb = (mapsb == hp.UNSEEN)
unseen_pos = (mapsb_pos == hp.UNSEEN)
unseen_neg = (mapsb_neg == hp.UNSEEN)


#### Calibrate as measured maps
max_meas = 18351.48
sig_meas = 912.88
# max_meas = np.nanmax(map_data)
# sig_meas = np.nanstd(map_data)
newmapsb[~unseen_sb] = mapsb[~unseen_sb]*max_meas/np.max(mapsb[~unseen_sb])# + np.random.randn((~unseen_sb).sum())*sig_meas
newmapsb_pos[~unseen_pos] = mapsb_pos[~unseen_pos]*max_meas/np.max(mapsb[~unseen_sb]) + np.random.randn((~unseen_pos).sum())*sig_meas*np.sqrt(2)
newmapsb_neg[~unseen_neg] = mapsb_neg[~unseen_neg]*max_meas/np.max(mapsb[~unseen_sb]) + np.random.randn((~unseen_neg).sum())*sig_meas*np.sqrt(2)


newmapdiff = newmapsb_pos - newmapsb_neg
newmapdiff[unseen] = hp.UNSEEN
plt.figure()
hp.gnomview(newmapsb, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,1), title='Both scans', min=-3e3, max=np.max(newmapsb[~unseen]))
hp.gnomview(newmapsb_pos, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,2), title='+ scans', min=-3e3, max=np.max(newmapsb[~unseen]))
hp.gnomview(newmapsb_neg, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,3), title='- scans', min=-3e3, max=np.max(newmapsb[~unseen]))
hp.gnomview(newmapdiff, rot=[np.mean(newazt), np.mean(newelt)], reso=12, sub=(1,4,4), title='Diff scans', min=-3e3, max=np.max(newmapsb[~unseen]))
plt.show()


In [ ]:
import fitting as fit
xs = 301
reso = 5

mymap = newmapsb.copy()
# mymap[mymap==hp.UNSEEN] = 0
# mymap /= np.max(mymap)
# # rot = [3, 1]
# rot = [0, 0]
# img = hp.gnomview(mymap, rot=rot, reso=reso, sub=(1,2,1), title='Az/El wrt Moon FP centered [QP #{} ; QS #{}]'.format(TESNum, indexQS+1), return_projected_map=True, xsize=xs,no_plot=True)
# img += np.random.randn(xs,xs)/100
# x = (np.arange(xs)-(xs-1)/2)*reso/60
# y = x.copy()
# m, fitted = fitgauss_img(img, x, y, doplot=True, nsig=3, mytit='Simulation', guess=[1,-0.5,-1,0.9])
poselt = 90 - np.degrees(pos_det_thetas[indexQS]) - deltelt # theta is elt
posazt = pmf.centre_coord_at_0(np.degrees(pos_det_phis[indexQS])) - deltazt # phi is azt
guess = [1e3, poselt, posazt, 1.03/pmf.conv_reso_fwhm]
print(guess)
m, mapxy, fitted, limits, fig_axs = pmf.fit_one_tes(mymap, xs, reso, rot=np.array([0., 0., 0.]), doplot=True, verbose=False, guess=guess, distok=3, mytit='', return_images=True, ms=10, renorm=True)
print(m)

## Get good maps and Moon positions on TES in Moon coordinates

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'
mydatadir2 = "/Users/huchet/Documents/code/scripts/MoonProject/"
mydatadir3 = "/Users/huchet/Documents/code/data/"

In [ ]:
%%script echo skipping
raw = False
RADec = False
nside = 256
azel = "v1"

if raw: raw_str = "_raw"
else: raw_str = ""
if RADec: RADec_str = "_RADec"
else: RADec_str = ""
if nside == 128: nside_str="_nside128"
else: nside_str = ""
if azel == "v1": azel_str = ""
else: azel_str = "_azel{}".format(azel)

filename = "202506-allmaps_moonpos-Jan14-2022_unturned{}{}{}{}.pkl".format(RADec_str, raw_str, nside_str, azel_str)
allTESNum, allmaps, _, visibly_ok_arr = pickle.load( open( mydatadir2 + filename, "rb" ) )
# filename_maps = "202506-allmaps-Jan14-2022_unturned{}{}.pkl".format(RADec_str, raw_str) # just the maps recomputed to try to get ht visibly not ok maps back
# allTESNum, allmaps = pickle.load( open( mydatadir2 + filename_maps, "rb" ) )
filename_fit = "202506-allmaps_moonpos-Jan14-2022_unturned{}{}.pkl".format(RADec_str, azel_str)
_, _, moon_fit, _ = pickle.load( open( mydatadir2 + filename_fit, "rb" ) )

# allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )
# allTESNum, allmaps, _, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_raw.pkl", "rb" ) )
# allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_RADec.pkl", "rb" ) )

In [ ]:
# %%script echo skipping
filename = "202506-allmaps_moonpos-Jan14-2022_flippedTESNum.pkl"
# filename = "202506-allmaps_moonpos-Jan14-2022_quad2TESNum.pkl"
# filename = "202506-allmaps_moonpos-Jan14-2022_unturned_flippedTESNum_azelv4.pkl"
allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + filename, "rb" ) )

In [ ]:
plt.figure()
hp.gnomview(mapsb, reso=10, rot=[0, 0], title='Az/El wrt Moon FP centered [QP #{} ; QS #{}]'.format(TESNum, indexQS+1))
plt.show()


## Fit the Moon on all TES simulations

In [ ]:
%%script echo skipping
reso = 5 #4 #5
xs = 301

allamp_sim = np.zeros(len(allTESNum))
allerramp_sim = np.zeros(len(allTESNum))
allFWHM_sim = np.zeros(len(allTESNum)) * np.nan
allerrFWHM_sim = np.zeros(len(allTESNum))
allxy_sim = np.zeros((len(allTESNum), 2)) * np.nan
allerrxy_sim = np.zeros((len(allTESNum), 2))
moon_fit_sim = []
thermometers = [4, 36, 68, 100, 132, 164, 196, 228]
i_QS_list = []

for i_TES in range(len(allTESNum)):
    if i_TES + 1 in thermometers:
        moon_fit_sim.append([[np.nan, np.nan], np.nan, np.nan])
        continue
    i_QS = pmf.iQP2iQS(i_TES)
    i_QS_list.append(i_QS)
    
    tod_i = TOD[i_QS, :]
    mapsb_i, mapcount_i = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], tod_i[scantype != 0], nside=nside)
    unseen_sb_i = (mapsb_i == hp.UNSEEN)
    mapsb_i[~unseen_sb_i] *= max_meas/np.max(mapsb_i[~unseen_sb_i]) # careful not to multiply the hp.UNSEEN values by the factor!
    poselt_i = pmf.centre_coord_at_0(90 - np.degrees(pos_det_thetas[i_QS]) - deltelt)
    posazt_i = pmf.centre_coord_at_0(np.degrees(pos_det_phis[i_QS]) - deltazt)
    guess = [np.max(mapsb_i), poselt_i, posazt_i, 1.03/pmf.conv_reso_fwhm]
    print(guess)
    # m, mapxy, fitted, limits, fig_axs = pmf.fit_one_tes(mapsb_i, xs, reso, rot=np.array([0., 0., 0.]), doplot=True, verbose=False, guess=guess, distok=3, mytit='', return_images=True, ms=10, renorm=True)
    m = pmf.fit_one_tes(mapsb_i, xs, reso, rot=np.array([0., 0., 0.]), doplot=False, verbose=False, guess=guess, distok=3, ms=10, renorm=True)
    # print(m)
    allFWHM_sim[i_TES] = m.values[3] * pmf.conv_reso_fwhm
    allerrFWHM_sim[i_TES] = m.errors[3] * pmf.conv_reso_fwhm
    allamp_sim[i_TES] = m.values[0] * pmf.conv_reso_fwhm
    allerramp_sim[i_TES] = m.errors[0] * pmf.conv_reso_fwhm
    allxy_sim[i_TES, :] = [m.values[2], m.values[1]] # m.values contains amplitude, elevation, azimuth, sigma
    allerrxy_sim[i_TES, :] = [m.errors[2], m.errors[1]]
    print('TES#{0}: FWHM = {1:5.2f}'.format(i_TES + 1, m.values[3] * pmf.conv_reso_fwhm))
    moon_fit_sim.append([allxy_sim[i_TES, :], allFWHM_sim[i_TES], allamp_sim[i_TES]])

In [ ]:
%%script echo skipping
diff_ = np.zeros((len(allTESNum), 2)) + np.nan
for ii in range(len(allTESNum)):
    if not np.any(np.isnan(moon_fit[ii][0])) and not np.any(np.isnan(moon_fit_sim[ii][0])):
        diff_[ii] = np.mod(np.array(moon_fit[ii][0]) - np.array(moon_fit_sim[ii][0]) - 180, 360) - 180
        print("TES {} diff = ".format(ii + 1), diff_[ii])

moon_pos = np.array([moon_fit[ii][0] for ii in range(len(allTESNum))])
moon_pos_sim = np.array([moon_fit_sim[ii][0] for ii in range(len(allTESNum))])

fig, ax = plt.subplots()
plt.subplot().set_aspect(1)
sim_scat = plt.scatter(np.mod(moon_pos_sim[:, 0] - 180, 360) - 180, moon_pos_sim[:, 1], c='k', label="Simulated positions")
plt.scatter(np.mod(moon_pos[:, 0] - 180, 360)- 180, moon_pos[:, 1], c='r', label="Real positions")
plt.xlabel('$az^{Moon}$')
plt.ylabel('$el^{Moon}$')
plt.legend()
plt.tight_layout()
all_names = allTESNum
pmp.hover_cursor(fig, ax, [sim_scat], ["sim_scat"], [all_names])
# plt.show()

In [ ]:
%%script echo skipping
plt.figure()
plt.scatter(diff_[:, 0], diff_[:, 1])
plt.xlabel("az diff")
plt.ylabel("el diff")
plt.show()

In [ ]:
# # test
# TESNum = 17
# iTES = TESNum - 1

# hp.gnomview(allmaps[iTES], rot=[0,0], reso=8, title='TES {}'.format(NumTES), min = -1e4, max=1e4)
# hp.gnomview(allmaps[iTES], rot=moon_fit[iTES][0], reso=10, title='TES {}'.format(NumTES), min = -1e4, max=1e4)
map_data = hp.gnomview(allmaps[iTES], rot=(0, 0), reso=10, sub=(1,3,1), title='Data TES {}'.format(TESNum), min = -1e4, max=1e4, no_plot=True, return_projected_map=True)
# rot = [4.6, 0.5, -2.5] # 152
# rot = [4.6, 1.3, 0]
rot = [0, 0]
map_simu = hp.gnomview(newmapsb, rot=rot, reso=10, sub=(1,3,2), title='Simu TES {}'.format(TESNum), min = -1e4, max=1e4, no_plot=True, return_projected_map=True)
fig, axs = plt.subplots(1, 3)
axs[0].imshow(map_data, vmin = -1e4, vmax=1e4)
axs[1].imshow(map_simu, vmin = -1e4, vmax=1e4)
axs[2].imshow(2*map_data - map_simu, vmin = -1e4, vmax=1e4)
plt.tight_layout()
plt.show()
print(moon_fit[iTES])

In [ ]:
delta_pos =  list(m.values[2:0:-1] - moon_fit[iTES][0])
print(delta_pos)
# [4.5658359258853585, 1.4750926931640054]
# [2.3469702337241207, 1.7129282474745677]

In [ ]:
print(TESNum)

In [ ]:
rot = delta_pos
print(rot)
map_simu = hp.gnomview(newmapsb, rot=rot, reso=10, sub=(1,3,2), title='Simu TES {}'.format(TESNum), min = -1e4, max=1e4, no_plot=True, return_projected_map=True)
fig, axs = plt.subplots(1, 3)
axs[0].imshow(map_data, vmin = -1e4, vmax=1e4)
axs[1].imshow(map_simu, vmin = -1e4, vmax=1e4)
axs[2].imshow(map_data - map_simu, vmin = -1e4, vmax=1e4)
plt.tight_layout()
plt.show()

In [ ]:
class map_sum_fit():
    def __init__(self, A):
        self.A = A.copy()
    def __call__(self, useless, x0):
        return np.ravel(np.einsum('ij,j', self.A, x0))


In [ ]:
def fit_Moon_spectrum(iTES, x0, data_maps, simu_TOD_nsub, axs=None, vmin=None, vmax=None, nsig=3): # indexQS = pmf.iQP2iQS(TESNum-1)
    # sum of all bands * band_weight = final map
    # fit on all TES at the same time?
    xs = 201
    reso = 10

    data = hp.gnomview(data_maps[iTES], rot=(0, 0), xsize=xs, reso=reso, title='Data TES {}'.format(TESNum), min = -1e4, max=1e4, no_plot=True, return_projected_map=True)

    nsubs = len(x0)
    # xs = len(data)
    
    scantype_hk, azt, elt, scantype, vmean = tdt.identify_scans(thk, qs.azimuth, qs.elevation, tt=qs.time, doplot=False, thr_speedmin=speedmin)
    # newazt = (qs.azimuth + azqubic - myazmoon) * np.cos(np.radians(qs.elevation)) # make it centered far from (0, 0)
    newazt = (qs.azimuth - myazmoon) * np.cos(np.radians(qs.elevation))
    newelt = (qs.elevation - myelmoon)

    ### get the gnomview back into a np.array in order to fit it
    badpix = data == hp.UNSEEN
    data[badpix] = 0          ### Set bad pixels to zero before returning the np.array()
    data[data**2 > 1e20] = 0
    print(np.min(data), np.max(data), np.std(data))

    A = np.empty((len(np.ravel(data)), nsubs))
    for i in range(nsubs):
        print(i)
        mapsb_i, mapcount_i = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], simu_TOD_nsub[i][indexQS, scantype != 0], nside=nside)
        A[:, i] = hp.gnomview(mapsb_i, rot=delta_pos, reso=reso, xsize=xs, title='Simu band_i TES {}'.format(TESNum), no_plot=True, return_projected_map=True).flatten()
        # A[:, i] = hp.gnomview(mapsb_i, rot=[0, 0], reso=reso, xsize=xs, title='Simu band_i TES {}'.format(TESNum), no_plot=True, return_projected_map=True).flatten()
        unseen_Ai = (A[:, i] == hp.UNSEEN)
        A[~unseen_Ai, i] = A[~unseen_Ai, i]*np.max(data)/np.max(A[~unseen_Ai, i])/12
    A[A == hp.UNSEEN] = 0
    # for i in range(nsubs):
    #     plt.figure()
    #     plt.imshow(A[:, i].reshape((xs, xs)))
    #     plt.show()

    spec_fit = map_sum_fit(A)

    xfit = np.ones_like(data) # useless but need to call functions

    ### Displays the image as an array
    ilimit = 150
    mm, ss = ft.meancut(data[:ilimit], nsig)
    ### Do the fit putting the UNSEEN to a very low weight
    errpix = xfit*0 + np.std(data)/10
    only_one_map = spec_fit(None, x0).reshape((xs, xs)) # if x0 is not zeros
    # only_one_map[only_one_map == 0] = 1
    # errpix = 1/(only_one_map**2)
    errpix[only_one_map < 1e-2 * np.max(only_one_map)] *= 1e5
    errpix[data==0] *= 1e3
    errpix[only_one_map.reshape((xs, xs))==0] *= 1e5
    iipos, jjpos = np.mgrid[0:xs, 0:xs]
    errpix[iipos > ilimit] *= 1e5

    errpix = np.ones_like(errpix) * 1
    errpix[iipos > ilimit] *= 1e5
    
    plt.figure()
    plt.imshow(errpix, vmin=np.min(errpix), vmax=np.min(errpix)*1e5)
    plt.show()


    fit_data = fit.Data(np.ravel(xfit), np.ravel(data), np.ravel(errpix), spec_fit)
    renorm = True # ou pas ?
    # lim = 1e5
    # m, ch2, ndf = fit_data.fit_minuit(guess=x0, limits=[[i, 1/lim*x0[i], lim*x0[i]] for i in range(len(x0))], renorm=renorm)
    m, ch2, ndf = fit_data.fit_minuit(guess=x0, limits=[[i, 0, 1e2] for i in range(len(x0))], renorm=renorm)

    print(m)

    ### Image of the fitted map
    fitted = np.reshape(spec_fit(xfit, m.values), (xs, xs))
    # fitted = A[:, i].reshape((xs, xs))
    
    vmin = -1e4
    vmax = 5e4
    # vmin = np.min(fitted)
    # vmax = np.max(fitted)
    origin = "upper"
    if vmin is None:
        vmin = mm-nsig*ss
    if vmax is None:
        vmax = np.max(data)
    fig, axs = plt.subplots(1, 4, width_ratios=(1, 1, 1, 0.05), figsize=(16, 5))
    axs[1].imshow(fitted, origin=origin, vmin=vmin, vmax=vmax)
    im = axs[2].imshow(data - fitted, origin=origin, vmin=vmin, vmax=vmax)
    axs[0].set_ylabel('Elevation [degrees]')
    for i in range(3):
        axs[i].set_xlabel('Azimuth [degrees]')
    axs[2].set_title('Residuals')

    print(np.mean(data))
    print(np.std(data))

    ax = axs[0]
    ax.clear()
    im = ax.imshow(data, origin=origin, vmin=vmin, vmax=vmax) # j = -azt = x, i = elt = y
    ax.set_xlabel('Degrees')
    ax.set_ylabel('Degrees')
    ax.legend()
    cax = axs[-1]
    cax.clear()
    plt.colorbar(im, cax=cax)
    return m, fitted, axs

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*')
print(dirs)
datadir = dirs[0]
ObsSite = pmf.get_ObsSite(name="salta")

In [ ]:
pixmoon = hp.query_disc(nside, uvmoon, moon_angsize, inclusive=True)
map_data_d = hp.gnomview(allmaps[iTES], rot=(0, 0), reso=10, sub=(1,3,1), title='Data TES {}'.format(TESNum), min = -1e4, max=1e4, no_plot=True, return_projected_map=True)
moon_spectrum_x0 = np.ones((d["n_nu_fit"])) # mononband not implemented
# moon_spectrum_x0 = (150/nus_in[:d["n_nu_fit"]])**4

In [ ]:
print(nus_in[:d["n_nu_fit"]])

In [ ]:
print(iTES)

In [ ]:
try:
    data_disk
except:
    data_disk = None
data_disk, tt, tinit, alltod, QPidx, azt, elt, newazt, newelt, scantype, Tbath = pmf.format_data(azqubic, start_tt=10000, ObsSite=ObsSite, speedmin=speedmin, data=data_disk, datadir=datadir)

moon_spectrum, map_fitted, axs = fit_Moon_spectrum(iTES, data_maps=allmaps, simu_TOD_nsub=TOD_nsub, x0=moon_spectrum_x0)

In [ ]:
print(moon_spectrum.values)
print(moon_spectrum_x0)

In [ ]:
order_0_pos_data = moon_fit[iTES][0]
order_0_pos_simu = m.values[2:0:-1]
# TES 67
if TESNum == 67:
    order_1_pos_end_data = [4.325, 1.415]
    # order_0_pos_simu = [2.982, 7.986]
    order_1_pos_end_simu = [9.5, 2.05]
elif TESNum == 152:
    order_1_pos_end_data = [-7.97, 7.1]
    # order_0_pos_simu = [3.2, 1.25]
    order_1_pos_end_simu = [-3.08, 7.65]

elif TESNum == 209:
    order_1_pos_end_data = [-3.86, -3.12]
    order_1_pos_end_simu = [0.41, -2.06]

list_sph_pos =  []
for pos in [order_0_pos_data, order_1_pos_end_data, order_0_pos_simu, order_1_pos_end_simu]:
    list_sph_pos.append(np.array([np.radians(90 - pos[1]), np.radians(pos[0])]))

npoints = 500
dist_order_0_1_deg = 10
delta_angle_deg = 1

In [ ]:
vals_data, theta_gc_data, phi_gc_data, angle_array_data = pmf.get_values_line(allmaps[iTES], list_sph_pos[0], list_sph_pos[1], npoints=npoints, ang_dist_deg=dist_order_0_1_deg, delta_angle_deg=delta_angle_deg)
vals_simu, theta_gc_simu, phi_gc_simu, angle_array_simu = pmf.get_values_line(newmapsb, list_sph_pos[2], list_sph_pos[3], npoints=npoints, ang_dist_deg=dist_order_0_1_deg, delta_angle_deg=delta_angle_deg)

# vals_data == np.where(vals_data**2>1e25, np.full_like(vals_data, np.NaN), vals_data)
vals_data[vals_data**2>1e25] = 0

fig, ax = plt.subplots()
ax.plot(angle_array_data, vals_data/np.max(vals_data))
ax.plot(angle_array_simu, vals_simu/np.max(vals_simu))
plt.show()

In [ ]:
hp.gnomview(allmaps[iTES], reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
hp.projscatter(theta_gc_data, phi_gc_data, marker='.', c="g", lonlat=False)
plt.show()
hp.gnomview(newmapsb, reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
hp.projscatter(theta_gc_simu, phi_gc_simu, marker='.', c="g", lonlat=False)
plt.show()

In [ ]:
from matplotlib.widgets import Button, Slider
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

class sliders_plot_sb():

    def __init__(self, in_map, synthbeam):
        self.reso = 5
        self.xs = 301
        self.nside = int(np.sqrt(len(in_map)/12))
        self.synthbeam = synthbeam

        mm = in_map#.copy()
        badpix = mm == hp.UNSEEN
        mm[badpix] = 0          ### Set bad pixels to zero before returning the np.array()
        self.in_map_proj = hp.gnomview(mm, reso=self.reso, rot=[0, 0, 0], return_projected_map=True, xsize=self.xs, no_plot=True).data
        # self.in_map = in_map.copy()

        self.init_pos()
        self.create_fig()
        self.add_sliders()
        self.init_plot()
        self.get_reset()

        # register the update function with each slider
        for slider in self.list_sliders:
            slider.on_changed(self.update)

        self.button.on_clicked(self.reset)

        self.reset(True)

    # Define initial parameters

    def init_pos(self):
        self.rot = np.array([4.6, 0.5, -2.5]) # TES 152?
        self.init_translation = self.rot[:-1]
        self.init_rotation = self.rot[-1]
        self.init_amplitude = 1

    def create_fig(self):
        # Create the figure and the line that we will manipulate
        self.fig, self.axs = plt.subplots(1, 3, figsize=(14, 6))
        self.fig.suptitle("TES {}".format(TESNum))

    def init_plot(self):
        self.update_sb_map()

        ax = self.axs[0]
        self.img0 = ax.imshow(self.in_map_proj, vmin=-1e-4, vmax=1e4)

        ax = self.axs[1]
        self.img1 = ax.imshow(self.sb_map, vmin=-1e-4, vmax=1e4)
        # self.img1 = ax.imshow(self.in_map_proj)

        ax = self.axs[2]
        self.img2 = ax.imshow(self.in_map_proj - self.sb_map, vmin=-1e-4, vmax=1e4)
        # self.img2 = ax.imshow(self.in_map_proj)

    def add_sliders(self):
        find_pos = True
        delta_deg = 10
        # adjust the main plot to make room for the sliders
        self.fig.subplots_adjust(left=0.25, bottom=0.25)

        # Make two horizontal slider to control the translation.
        # axtransaz = self.fig.add_axes([0.25, 0.15, 0.65, 0.03])
        # axtransel = self.fig.add_axes([0.25, 0.1, 0.65, 0.03])

        self.transaz_slider = self.init_rot_slider(init_pos=self.init_translation[0], delta_deg=delta_deg,
                                                    axpos=[0.25, 0.15, 0.65, 0.03], find_pos=find_pos, orientation="horizontal",
                                                    label="Rot1 [deg]")

        self.transel_slider = self.init_rot_slider(init_pos=self.init_translation[1], delta_deg=delta_deg,
                                                    axpos=[0.25, 0.1, 0.65, 0.03], find_pos=find_pos, orientation="horizontal",
                                                    label="Rot2 [deg]")


        # Make a vertically oriented slider to control the amplitude
        axamp = self.fig.add_axes([0.15, 0.25, 0.0225, 0.63])
        self.amp_slider = Slider(
            ax=axamp,
            label="Scale",
            valmin=0.2,
            valmax=20,
            valinit=self.init_amplitude,
            orientation="vertical"
        )


        # Make a vertically oriented slider to control the rotation
        self.rotation_slider = self.init_rot_slider(init_pos=self.init_rotation, delta_deg=delta_deg,
                                                    axpos=[0.1, 0.25, 0.0225, 0.63], find_pos=find_pos, orientation="vertical",
                                                    label="Rot3 [deg]")

        self.list_sliders = [self.transaz_slider, self.transel_slider, self.amp_slider, self.rotation_slider]

    
    def init_rot_slider(self, init_pos, delta_deg, axpos, find_pos, orientation, label):
        if find_pos:
            vals = [-180, 180]
        else:
            vals = [init_pos - delta_deg, init_pos + delta_deg]
        axrot = self.fig.add_axes(axpos)
        slider = Slider(
            ax=axrot,
            label=label,
            valmin=vals[0],
            valmax=vals[1],
            valinit=init_pos,
            orientation=orientation
        )
        return slider

    def update_sb_map(self):
        self.sb_map = hp.gnomview(self.synthbeam, reso=self.reso, rot=[self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val], return_projected_map=True, xsize=self.xs, no_plot=True).data * self.amp_slider.val

    # The function to be called anytime a slider's value changes
    def update(self, val):
        self.update_sb_map()
        self.img1.set_data(self.sb_map)
        self.img2.set_data(self.in_map_proj - self.sb_map)
        self.fig.canvas.draw_idle()

    def get_reset(self):
        # Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
        resetax = self.fig.add_axes([0.8, 0.025, 0.1, 0.04])
        self.button = Button(resetax, 'Reset', hovercolor='0.975')


    def reset(self, event):
        for slider in self.list_sliders:
            slider.reset()



In [ ]:
# mynum = 133
# idx = np.where(np.array(allTESNum)==mynum)[0][0]

sliders_plot_sb(allmaps[iTES], newmapsb)

In [ ]:
for fp_idx in range(len(FPidentity)):
    # if FPidentity[fp_idx].TES==0:
    #     continue
    if FPidentity[fp_idx].quadrant != 2:
        continue
    print(FPidentity[fp_idx].QPindex)

In [ ]:
import qubicpack as qp
FPidentity = qp.pixel_translation.make_id_focalplane()

In [ ]:
QPidx = np.array([FPidentity[fp_idx].QPindex for fp_idx in range(len(FPidentity)) if FPidentity[fp_idx].quadrant == 3]).reshape(17, 17) # TD quadrant = 3

In [ ]:
print(QPidx)

In [ ]:
print(np.flip(np.flip(QPidx, axis=0).T, axis=0))